In [1]:
# ----------------------------------------------------
# 1. 라이브러리 로드 및 모델 초기화
# ----------------------------------------------------
import torch
import numpy as np
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import gdown
import cv2
import os 

# 사용할 모델 정의
CLIP_MODEL_NAME = "openai/clip-vit-base-patch32"
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Loading CLIP Model: {CLIP_MODEL_NAME} on device: {device}")


# hugging face
HF_TOKEN = os.getenv("HUGGINGFACE_TOKEN")

try:
    processor = CLIPProcessor.from_pretrained(CLIP_MODEL_NAME, use_fast=True, use_auth_token=HF_TOKEN) 
    model = CLIPModel.from_pretrained(CLIP_MODEL_NAME, use_auth_token=HF_TOKEN).to(device)
    print("CLIP 모델 로드 성공.")
except Exception as e:
    if "Unauthorized" in str(e):
        print("\n[오류 해결 안내] 401 Unauthorized 오류가 발생했습니다. Hugging Face 토큰 인증이 필요합니다.")
        print("1. 터미널에서 'export HUGGINGFACE_TOKEN=\"당신의 토큰\"' 설정 후 다시 실행하거나,")
        print("2. 'huggingface-cli login'을 실행하여 토큰을 입력해주세요.")
    else:
        # torchvision 누락 오류 등 다른 오류가 발생할 경우를 대비하여 메시지를 남김
        print(f"CLIP 모델 로드 실패. 라이브러리 설치 또는 기타 오류를 확인하세요: {e}")
    exit()

# ----------------------------------------------------
# 2. 한국 문화 객체 목록 정의 (검증 DB의 K_Object 테이블과 매핑됨)
# ----------------------------------------------------

# CLIP이 인식해야 할 한국 문화 객체 목록
# 이 목록은 향후 Cloud SQL의 K_Object 테이블에서 불러와야 합니다.
K_CULTURE_OBJECTS = [
    "Korean traditional hat, called Gat", 
    "Korean traditional jacket, called Jeogori",
    "Korean traditional trousers, called Baji",
    "Modern smartphone", 
    "A person wearing a traditional Hanbok",
    "A contemporary concrete building",
    "A person wearing modern sneakers",
]

# 프롬프트 구성: CLIP의 Zero-Shot 성능을 극대화하기 위해 템플릿 사용
# 'a photo of [객체]' 형태가 가장 일반적입니다.
candidate_labels = [f"a photo of {obj}" for obj in K_CULTURE_OBJECTS]

# ----------------------------------------------------
# 3. 이미지 준비 (GCS에서 파일 가져오기 시뮬레이션)
# ----------------------------------------------------

# 테스트 이미지 로드 (예시: 로컬 파일 경로)
# 실제 프로젝트에서는 GCS에서 파일을 직접 읽어와야 합니다.
try:
    # 예시 이미지 파일 경로를 설정하세요. (Graduation_Project_1 폴더 안에 저장된 테스트 이미지)
    image_path = "images/sample_hanbok_with_sneakers.jpg" 
    image = Image.open(image_path).convert("RGB")
    print(f"\n이미지 로드 성공: {image_path} ({image.size[0]}x{image.size[1]})")
except FileNotFoundError:
    print("\n[오류] 테스트 이미지를 찾을 수 없습니다. 프로젝트 폴더에 이미지를 저장해 주세요.")
    exit()

# ----------------------------------------------------
# 4. Zero-Shot Classification 실행
# ----------------------------------------------------
print("\n--- 객체 인식 ---")

# 프로세서로 이미지와 텍스트를 모델 입력 형태로 변환
inputs = processor(text=candidate_labels, images=image, return_tensors="pt", padding=True).to(device)

# 모델 추론 실행
with torch.no_grad():
    outputs = model(**inputs)
    
# 로짓(logit)을 확률로 변환
logits_per_image = outputs.logits_per_image  
probs = logits_per_image.softmax(dim=1).cpu().numpy()[0]

# 결과 정리 (객체 이름과 확률)
results = []
for i, prob in enumerate(probs):
    results.append({
        "object": K_CULTURE_OBJECTS[i],
        "probability": prob
    })

# 확률이 높은 순으로 정렬
results.sort(key=lambda x: x["probability"], reverse=True)

# ----------------------------------------------------
# 5. 최종 결과 출력 및 DB 매핑 준비
# ----------------------------------------------------
print("\n--- 인식 결과 (상위 5개 객체) ---")
for i, result in enumerate(results[:5]):
    print(f"순위 {i+1}: {result['object']} (확률: {result['probability']:.4f})")
    
# **[다음 단계 준비]**
# 1. 특정 임계값(Threshold, 예: 0.8 이상)을 넘는 객체만 최종 객체 목록으로 확정합니다.
FINAL_RECOGNIZED_OBJECTS = [
    result['object'] for result in results if result['probability'] > 0.8
]

#print("\n--- ---")
print(FINAL_RECOGNIZED_OBJECTS)


Loading CLIP Model: openai/clip-vit-base-patch32 on device: cuda
CLIP 모델 로드 성공.

이미지 로드 성공: images/sample_hanbok_with_sneakers.jpg (550x413)

--- 객체 인식 ---

--- 인식 결과 (상위 5개 객체) ---
순위 1: Korean traditional jacket, called Jeogori (확률: 0.4240)
순위 2: A person wearing a traditional Hanbok (확률: 0.3172)
순위 3: Korean traditional trousers, called Baji (확률: 0.2259)
순위 4: Korean traditional hat, called Gat (확률: 0.0326)
순위 5: Modern smartphone (확률: 0.0002)
[]


In [5]:
# ----------------------------------------------------
# 1. 라이브러리 로드 및 모델 초기화 (CLIP 부분은 잠시 주석 처리)
# ----------------------------------------------------
import torch
import numpy as np
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import gdown
import cv2
import os 
from ultralytics import YOLO # YOLOv8 라이브러리 추가

# ----------------------------------------------------
# 2. YOLOv8 모델 로드
# ----------------------------------------------------
print("Loading YOLOv8s Model for Object Detection...")
try:
    # 'yolov8s.pt'는 작고 빠른 YOLOv8의 사전 학습된 모델입니다.
    # GPU가 있다면 자동으로 사용됩니다.
    yolo_model = YOLO('yolov8s.pt')
    print("YOLOv8 모델 로드 성공.")
except Exception as e:
    print(f"[오류] YOLOv8 모델 로드 실패: {e}")
    exit()

# ----------------------------------------------------
# 3. 이미지 준비 및 객체 탐지 실행
# ----------------------------------------------------

def run_yolo_detection(image_path: str):
    """
    YOLOv8을 실행하여 이미지에서 객체를 탐지하고 결과를 반환합니다.
    """
    try:
        # conf=0.25 (확률 25% 이상 객체만 탐지), iou=0.7 (중복 박스 제거 기준)
        results = yolo_model.predict(image_path, conf=0.1, iou=0.7, save=False, verbose=False)
        
        detected_objects = []
        
        # 탐지 결과에서 Bounding Box와 클래스 이름 추출
        for result in results:
            if result.boxes:
                for box in result.boxes:
                    # BBox 좌표 (x1, y1, x2, y2)를 정수형으로 변환
                    x1, y1, x2, y2 = [int(x) for x in box.xyxy[0].tolist()]
                    conf = box.conf[0].item() # 확률
                    class_id = int(box.cls[0].item())
                    
                    # YOLOv8의 기본 클래스 이름 가져오기 (예: 'person', 'shoe' 등)
                    class_name = yolo_model.names[class_id] 
                    
                    detected_objects.append({
                        "class_name": class_name,
                        "bbox": [x1, y1, x2, y2],
                        "confidence": conf,
                        # 다음 단계: 객체 분리 후 CLIP 분류를 위한 자리
                        "clip_verified_name": None 
                    })

        return detected_objects

    except Exception as e:
        print(f"[오류] YOLO 탐지 실행 중 문제 발생: {e}")
        return []

# ----------------------------------------------------
# 4. 테스트 실행
# ----------------------------------------------------
if __name__ == "__main__":
    image_path = "images/sample_hanbok_with_sneakers.jpg"
    
    print(f"\n--- YOLOv8 객체 탐지 시작 ({image_path}) ---")
    detected_objects = run_yolo_detection(image_path)
    
    if not detected_objects:
        print("[실패] 탐지된 객체가 없습니다. 이미지 경로와 파일 상태를 확인하세요.")
        exit()
        
    print(f"\n[성공] 총 {len(detected_objects)}개의 객체 탐지 완료.")
    
    # 결과 출력
    for i, obj in enumerate(detected_objects):
        print(f"객체 {i+1}: {obj['class_name']} (확률: {obj['confidence']:.2f}, BBox: {obj['bbox']})")
        
    print("\n--- 다음 단계: 각 객체 조각을 CLIP으로 정밀 분류 ---")
    # 여기서 각 객체의 BBox 정보를 사용하여 이미지를 잘라낸 후,
    # 잘라낸 조각을 CLIP에 넣어 'Sneakers'인지 'Jipsin(짚신)'인지 분류해야 합니다.

#``eof

## 2. 다음 단계: Two-Stage Verification

#위 코드를 실행하여 **YOLOv8이 이미지에서 'person', 'shoe' 같은 일반적인 객체를 잘 찾아내는지** 확인해 주세요.

#1.  **YOLO 탐지:** YOLOv8이 이미지에서 **'shoe' (신발)** 객체를 찾아내고 그 위치(BBox)를 반환할 것입니다.
#2.  **객체 조각 분리:** 반환된 BBox 좌표를 사용하여 원본 이미지에서 **신발 부분만 잘라냅니다.** 
#3.  **CLIP 정밀 분류:** 잘라낸 신발 이미지를 CLIP 모델에 넣고, 더 상세한 한국 문화 객체 목록(예: **`Modern Sneakers`**, **`Korean Jipsin`**)과 비교하여 최종적으로 **'현대 스니커즈'**임을 확정합니다.

#이 과정을 거쳐야 비로소 **`Modern Sneakers`** 객체가 최종 검증 모듈로 전달되고, 고증 오류 점수 100점을 받을 수 있게 됩니다.

#YOLOv8 탐지 결과가 성공적으로 나오는지 확인하신 후, **CLIP 정밀 분류를 통합하는 다음 코드**를 구현해 드리겠습니다.

Loading YOLOv8s Model for Object Detection...
YOLOv8 모델 로드 성공.

--- YOLOv8 객체 탐지 시작 (images/sample_hanbok_with_sneakers.jpg) ---

[성공] 총 4개의 객체 탐지 완료.
객체 1: person (확률: 0.86, BBox: [312, 178, 726, 1002])
객체 2: person (확률: 0.29, BBox: [785, 539, 806, 561])
객체 3: person (확률: 0.15, BBox: [761, 540, 779, 561])
객체 4: person (확률: 0.12, BBox: [809, 536, 828, 562])

--- 다음 단계: 각 객체 조각을 CLIP으로 정밀 분류 ---


In [11]:
# ----------------------------------------------------
# 1. 라이브러리 로드 및 모델 초기화
# ----------------------------------------------------
import torch
import numpy as np
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
from ultralytics import YOLO
import os 
from typing import List, Dict, Any # 타입 힌트용 라이브러리 추가

# 모델 설정
CLIP_MODEL_NAME = "openai/clip-vit-base-patch32"
device = "cuda" if torch.cuda.is_available() else "cpu"
HF_TOKEN = os.getenv("HUGGINGFACE_TOKEN")

print(f"Loading CLIP and YOLO Models on device: {device}")

# CLIP 모델 로드
try:
    processor = CLIPProcessor.from_pretrained(CLIP_MODEL_NAME, use_fast=True, use_auth_token=HF_TOKEN) 
    model = CLIPModel.from_pretrained(CLIP_MODEL_NAME, use_auth_token=HF_TOKEN).to(device)
    print("CLIP 모델 로드 성공.")
except Exception as e:
    if "Unauthorized" in str(e):
        print("\n[오류] 401 Unauthorized 오류: Hugging Face 토큰 인증 필요.")
    else:
        print(f"[CLIP 오류] 모델 로드 실패: {e}")
    exit()

# YOLOv8 모델 로드
try:
    yolo_model = YOLO('yolov8s.pt')
    print("YOLOv8 모델 로드 성공.")
except Exception as e:
    print(f"[YOLO 오류] 모델 로드 실패: {e}")
    exit()

# ----------------------------------------------------
# 2. 정밀 분류 프롬프트 정의 (신발/의복 특화)
# ----------------------------------------------------

# CLIP이 정밀 분류해야 할 하위 객체 목록
# 이 목록은 고증 오류를 판단하는 데 필수적인 항목들입니다.
K_CULTURE_FINE_GRAINED_OBJECTS = [
    # 신발 관련 프롬프트 (오류 탐지 핵심)
    "a photo of Korean traditional straw shoes (Jipsin)",
    "a photo of Korean traditional ceremonial shoes",
    "a photo of modern, white running sneakers", 
    "a photo of a modern casual shoe",
    "a photo of the ground or a rock", # 배경일 가능성 대비

    # 의복 관련 프롬프트 (전통 의복 확정)
    "a photo of a high-end silk Hanbok robe", 
    "a photo of a common modern garment",
]

# 프롬프트 구성
fine_grained_labels = [f"This object is {obj}" for obj in K_CULTURE_FINE_GRAINED_OBJECTS]

# ----------------------------------------------------
# 3. 객체 탐지 (YOLO) 및 정밀 분류 (CLIP) 통합 함수
# ----------------------------------------------------

def two_stage_verify(image_path: str) -> List[Dict[str, Any]]:
    """
    1단계: YOLO로 person 객체 탐지
    2단계: 탐지된 person BBox를 잘라내어 CLIP으로 신발 영역 정밀 분석
    """
    original_image = Image.open(image_path).convert("RGB")
    final_recognized_objects = []
    
    # --- 1단계: YOLO 객체 탐지 및 BBox 추출 ---
    # conf=0.25 임계값을 유지하여 'person' 객체를 찾습니다.
    yolo_results = yolo_model.predict(image_path, conf=0.25, iou=0.7, save=False, verbose=False)
    person_bboxes = []

    for result in yolo_results:
        if result.boxes:
            for box in result.boxes:
                class_name = yolo_model.names[int(box.cls[0].item())]
                if class_name == 'person':
                    person_bboxes.append([int(x) for x in box.xyxy[0].tolist()])

    print(f"\n[1단계 결과] 총 {len(person_bboxes)}개의 'person' 객체 탐지.")

    # --- 2단계: CLIP 정밀 분류 및 오류 객체 추출 ---
    for bbox in person_bboxes:
        x1, y1, x2, y2 = bbox # 사람 전체 BBox
        
        # [신발 영역(하단 1/4) 강제 추출 로직]
        # 사람 전체 영역의 하단 75% 지점부터 신발 영역으로 간주하고 잘라냅니다.
        crop_y1 = y1 + int( (y2 - y1) * 0.75 ) 
        shoe_bbox = [x1, crop_y1, x2, y2]
        shoe_crop = original_image.crop(shoe_bbox)
        
        # --- CLIP 정밀 분류 실행 (신발 조각) ---
        inputs_shoe = processor(text=fine_grained_labels, images=shoe_crop, return_tensors="pt", padding=True).to(device)

        with torch.no_grad():
            outputs_shoe = model(**inputs_shoe)
            probs_shoe = outputs_shoe.logits_per_image.softmax(dim=1).cpu().numpy()[0]
             
        # 결과 해석 및 최고 확률 객체 추출
        shoe_results = [{"object": K_CULTURE_FINE_GRAINED_OBJECTS[i], "probability": probs_shoe[i]} for i in range(len(probs_shoe))]
        shoe_results.sort(key=lambda x: x["probability"], reverse=True)
        top_shoe = shoe_results[0]
        
        # --- 최종 목록에 고증 오류 객체 및 전통 복식 객체 추가 ---
        
        # 1. 고증 오류 객체 확정 (스니커즈 탐지 및 임계값 0.6 설정)
        if "running sneakers" in top_shoe['object'] and top_shoe['probability'] > 0.6:
            final_recognized_objects.append({
                "name": "Modern sneakers (Anachronism)",
                "probability": top_shoe['probability'],
                "bbox": shoe_bbox
            })
            print(f"  -> [오류 감지 SUCCESS] {top_shoe['object']} 확정 (확률: {top_shoe['probability']:.4f})")
        else:
             print(f"  -> [정밀 분류] 신발 영역 최고 확률: {top_shoe['object']} (확률: {top_shoe['probability']:.4f})")


        # 2. 전통 복식 객체 추가 (고증 검증 쌍을 만들기 위해 필수)
        # 이 코드가 없으면 DB 검증 자체가 불가능합니다.
        # 사람 전체 BBox를 입력하여 '한복 로브'를 확정하는 로직을 재사용합니다.
        
        # 복식 정밀 분류 실행 (사람 전체 BBox 사용)
        inputs_robe = processor(text=fine_grained_labels, images=original_image.crop(bbox), return_tensors="pt", padding=True).to(device)
        with torch.no_grad():
            outputs_robe = model(**inputs_robe)
            probs_robe = outputs_robe.logits_per_image.softmax(dim=1).cpu().numpy()[0]
        
        robe_results = [{"object": K_CULTURE_FINE_GRAINED_OBJECTS[i], "probability": probs_robe[i]} for i in range(len(probs_robe))]
        robe_results.sort(key=lambda x: x["probability"], reverse=True)
        top_robe = robe_results[0]
        
        if "Hanbok robe" in top_robe['object'] and top_robe['probability'] > 0.8:
             final_recognized_objects.append({
                "name": "Traditional Hanbok Robe",
                "probability": top_robe['probability'],
                "bbox": bbox 
            })


    return final_recognized_objects

# ----------------------------------------------------
# 4. 테스트 실행
# ----------------------------------------------------
if __name__ == "__main__":
    # 이 파일은 images/sample_hanbok_with_sneakers.jpg 이미지가 있다는 가정하에 실행됩니다.
    image_path = "images/sample_hanbok_with_sneakers.jpg" 
    
    try:
        final_objects = two_stage_verify(image_path)
    except FileNotFoundError:
        print("\n[오류] 테스트 이미지를 찾을 수 없습니다. 경로를 확인하세요.")
        exit()
        
    print("\n--- 고증 검증 모듈로 전달될 최종 객체 목록 ---")
    if not final_objects:
        print("[결과] 고증 검증 객체가 탐지되지 않았습니다.")
        exit()
        
    # 최종 결과 출력
    for obj in final_objects:
        print(f"객체: {obj['name']}, 확률: {obj['probability']:.4f}, BBox: {obj.get('bbox', 'N/A')}")
        
    print("\n--- 다음 단계: Cloud SQL DB 연동 및 점수 산출 ---")
#```eof

#이제 이 코드를 실행하면, **YOLO가 찾은 사람 영역의 하단(신발)에 대해 CLIP이 정밀 분류를 수행**하여, **`Modern sneakers (Anachronism)`** 객체를 높은 확률로 최종 목록에 포함시킬 것입니다.

#이 결과로 **'Traditional Hanbok Robe'**와 **'Modern sneakers'**라는 두 개의 불일치 객체가 생성되어, 다음 단계인 **Cloud SQL 기반의 고증 점수 산출**로 넘길 수 있는 완벽한 뼈대가 완성됩니다.

Loading CLIP and YOLO Models on device: cuda
CLIP 모델 로드 성공.
YOLOv8 모델 로드 성공.

[1단계 결과] 총 2개의 'person' 객체 탐지.
  -> [오류 감지 SUCCESS] a photo of modern, white running sneakers 확정 (확률: 0.9788)
  -> [정밀 분류] 신발 영역 최고 확률: a photo of modern, white running sneakers (확률: 0.3087)

--- 고증 검증 모듈로 전달될 최종 객체 목록 ---
객체: Modern sneakers (Anachronism), 확률: 0.9788, BBox: [312, 796, 726, 1002]
객체: Traditional Hanbok Robe, 확률: 0.9698, BBox: [312, 178, 726, 1002]

--- 다음 단계: Cloud SQL DB 연동 및 점수 산출 ---


In [ ]:
# (임시) DB 연동 예
import pymysql.cursors
import pandas as pd
from typing import List, Dict, Any

# 주의: 보안을 위해 이 정보는 실제 서비스에서는 환경 변수나 Secret Manager에서 불러와야 합니다.
DB_CONFIG = {
    'host': 'YOUR_CLOUD_SQL_IP',     # Cloud SQL 외부 IP 주소
    'user': 'your_db_user',           # DB 사용자 이름
    'password': 'your_db_password',   # DB 비밀번호
    'database': 'kculture_db',        # 데이터베이스 이름
    'port': 3306,                     # MySQL/MariaDB 기본 포트
    'cursorclass': pymysql.cursors.DictCursor
}

def get_db_connection():
    """DB 연결 객체를 생성하고 반환합니다."""
    try:
        connection = pymysql.connect(**DB_CONFIG)
        return connection
    except pymysql.Error as e:
        print(f"ERROR: Cloud SQL 접속 실패: {e}")
        return None

def get_object_info_and_rules(recognized_object_names: List[str]) -> List[Dict[str, Any]]:
    """
    인식된 객체 목록을 기반으로 DB에서 관련 규칙 및 정보를 조회합니다.
    """
    connection = get_db_connection()
    if not connection:
        return []

    # 1. 인식된 객체 이름 목록을 DB 쿼리용 튜플로 변환
    name_placeholders = ', '.join(['%s'] * len(recognized_object_names))
    
    # 2. SQL 쿼리: 인식된 객체 쌍과 관련된 모든 불일치 규칙 조회
    # 객체 이름을 기반으로 ID를 찾아 규칙 테이블과 조인합니다.
    sql_query = f"""
    SELECT 
        r.inconsistency_score, 
        o1.object_name AS object_a, 
        o2.object_name AS object_b,
        r.description
    FROM Consistency_Rule r
    JOIN K_Object o1 ON r.object_id_A = o1.object_id
    JOIN K_Object o2 ON r.object_id_B = o2.object_id
    WHERE o1.object_name IN ({name_placeholders}) 
      AND o2.object_name IN ({name_placeholders});
    """
    
    try:
        with connection.cursor() as cursor:
            # 쿼리 실행: name_placeholders에 인식된 객체 이름을 두 번 넣어 줍니다.
            params = recognized_object_names + recognized_object_names
            cursor.execute(sql_query, params)
            result = cursor.fetchall()
            return result
    except Exception as e:
        print(f"ERROR: DB 쿼리 실행 실패: {e}")
        return []
    finally:
        connection.close()

def calculate_anachronism_score(recognized_objects: List[str]) -> Dict[str, Any]:
    """
    인식된 객체 목록을 기반으로 고증 불일치 점수를 계산합니다.
    """
    all_rules = get_object_info_and_rules(recognized_objects)
    
    total_score = 0
    triggered_rules = []

    # 1. 모든 인식된 객체 쌍에 대해 규칙을 확인
    for rule in all_rules:
        # DB에서 가져온 규칙이 현재 인식된 객체 쌍에 정확히 적용되는지 확인 (불필요한 규칙 필터링)
        
        # 'Modern sneakers'와 'Traditional Hanbok Robe' 같은 쌍이 있는지 확인
        if rule['object_a'] in recognized_objects and rule['object_b'] in recognized_objects:
            total_score += rule['inconsistency_score']
            triggered_rules.append({
                "score": rule['inconsistency_score'],
                "objects": [rule['object_a'], rule['object_b']],
                "reason": rule['description']
            })
            
    return {
        "total_anachronism_score": total_score,
        "triggered_rules": triggered_rules,
        "judgment": "고증 오류 심각" if total_score >= 100 else ("경미한 오류" if total_score > 0 else "고증 적합")
    }

In [1]:
from groundingdino.util.inference import load_model, predict, annotate
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import torch, numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"

# ----------------------------------------------------
# 1️⃣ Grounding DINO 로드 (객체 탐지)
# ----------------------------------------------------
grounding_model = load_model("GroundingDINO_SwinT_OGC.pth", "config/GroundingDINO_SwinT_OGC.py")

# 탐지할 한국문화 객체 목록
PROMPT = "hanbok, gat, smartphone, building, person, shoes, jeogori, baji"

image = Image.open("images/sample_hanbok_with_sneakers.jpg").convert("RGB")
boxes, labels, scores = predict(
    model=grounding_model,
    image=image,
    caption=PROMPT,
    box_threshold=0.25,
    text_threshold=0.25,
)

# ----------------------------------------------------
# 2️⃣ CLIP 로드 (세밀 분류)
# ----------------------------------------------------
clip_model_name = "openai/clip-vit-base-patch32"
processor = CLIPProcessor.from_pretrained(clip_model_name)
clip_model = CLIPModel.from_pretrained(clip_model_name).to(device)

# fine-grained 객체 목록
CULTURE_LABELS = [
    "Traditional Korean hat (Gat)",
    "Traditional jacket (Jeogori)",
    "Traditional pants (Baji)",
    "Hanbok",
    "Modern sneakers",
    "Traditional straw shoes (Jipsin)",
    "Modern smartphone",
    "Concrete building"
]

# ----------------------------------------------------
# 3️⃣ 박스별 crop → CLIP 분류
# ----------------------------------------------------
results = []
for i, box in enumerate(boxes):
    cropped = image.crop(box)
    inputs = processor(text=CULTURE_LABELS, images=cropped, return_tensors="pt", padding=True).to(device)

    with torch.no_grad():
        outputs = clip_model(**inputs)
        probs = outputs.logits_per_image.softmax(dim=1).cpu().numpy()[0]

    top_idx = np.argmax(probs)
    results.append({
        "label": CULTURE_LABELS[top_idx],
        "confidence": probs[top_idx],
        "bbox": box,
        "base_label": labels[i],
        "dino_score": float(scores[i]),
    })

# ----------------------------------------------------
# 4️⃣ 결과 출력
# ----------------------------------------------------
for i, r in enumerate(results):
    print(f"[{i+1}] {r['label']} (CLIP: {r['confidence']:.2f}, DINO: {r['dino_score']:.2f}) BBox: {r['bbox']}")


ModuleNotFoundError: No module named 'groundingdino'

In [3]:
pip install git+https://github.com/IDEA-Research/GroundingDINO.git

  Cloning https://github.com/IDEA-Research/GroundingDINO.git to c:\users\public\documents\estsoft\creatortemp\pip-req-build-ai2c347k
  Resolved https://github.com/IDEA-Research/GroundingDINO.git to commit 856dde20aee659246248e20734ef9ba5214f5e44
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached addict-2.4.0-py3-none-any.whl.metadata (1.0 kB)
  Using cached supervision-0.26.1-py3-none-any.whl.metadata (13 kB)
  Using cached pycocotools-2.0.10-cp312-abi3-win_amd64.whl.metadata (1.3 kB)
Using cached supervision-0.26.1-py3-none-any.whl (207 kB)
Using cached addict-2.4.0-py3-none-any.whl (3.8 kB)
Using cached pycocotools-2.0.10-cp312-abi3-win_amd64.whl (76 kB)
  Running setup.py clean for groundingdino
Failed to build groundingdino
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/IDEA-Research/GroundingDINO.git 'C:\Users\Public\Documents\ESTsoft\CreatorTemp\pip-req-build-ai2c347k'
  DEPRECATION: Building 'groundingdino' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'groundingdino'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [63 lines of output]
  Compiling with CUDA
  running bdist_wheel
  C:\Users\jech0\anaconda3\Lib\site-packages\torch\utils\cpp_extension.py:576: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninj

In [7]:
pip install opencv-python pillow matplotlib transformers


Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install timm


  Using cached timm-1.0.20-py3-none-any.whl.metadata (61 kB)
Using cached timm-1.0.20-py3-none-any.whl (2.5 MB)
Note: you may need to restart the kernel to use updated packages.
